In [1]:
author_list = [111, 59, 71, 21] 
doc_id = 1007


In [2]:
import StyloNeural as Stylo

In [3]:
(labels_index, predYList, predY, history) = Stylo.getResults(authorList=author_list, 
                                                             doc_id=2178, 
                                                             level = 'char', 
                                                             glove = '../../glove/',
                                                             nb_epoch = 30)

Using Theano backend.
Using gpu device 0: GeForce GTX 950 (CNMeM is disabled, cuDNN 5005)


Indexing char vectors.
Found 69 char vectors.
Processing text dataset
chunk_size 1449.
Execution completed
Read completed
Number of rows: 145
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (52542, 2)
Mean: 13135
Min: 5217
Max: 25228
Authors [21, 59, 71, 111].
Found 1200 texts.
Found 1200 labels.
Shape of data tensor: (1200, 1449, 69)
Shape of label tensor: (1200, 4)
Done compiling.
Train on 960 samples, validate on 240 samples
Epoch 1/30
960/960 [==============================] - 7s - loss: 1.3867 - acc: 0.2417 - val_loss: 1.3865 - val_acc: 0.2500

ValueError: 2 is not in list

In [ ]:
print predY

In [ ]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()